# Project 1 


In [2]:
# Required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from IPython.display import Image
import scipy.integrate
import random
import imageio.v2 as imageio

## Numerical experiments of Crank-Nicolson scheme

$
\left(I - \frac{r}{2}D\right)U^{n+1} = \left(I + \frac{r}{2}D +akI+\frac{ark}{2}D + \frac{a^2k^2}{2}I\right)U^{n}$

In [3]:
# implement the modified Crank Nicolson method defined by the equation
#\left(I - \frac{r}{2}D\right)U^{n+1} = \left(I + \frac{r}{2}D +akI+\frac{ark}{2}D + \frac{a^2k^2}{2}I\right)U^{n}
def f(x,a):
    return a*x

def crank_nicolson(a, r, k, T, U0, N):
    h = T/N
    t = np.linspace(0, T, N+1)
    U = np.zeros(N+1)
    U[0] = U0
    # create the matrix D
    D = np.zeros((N+1, N+1))
    for i in range(N+1):
        D[i,i] = 1
        if i > 0:
            D[i,i-1] = -1


    # create the matrix I
    I = np.zeros((N+1, N+1))
    for i in range(N+1):
        I[i,i] = 1

    # create the matrix A
    A = I + r/2*D + a*k*I + r*k/2*D + a**2*k**2/2*I

    # create the matrix B
    B = I - r/2*D

    for n in range(N):
        U[n+1] = np.linalg.solve(A, np.dot(B, U[n]))
    return t, U

# define the parameters
a = 1
r = 1
k = 0.1
T = 1
U0 = 1
N = 100

# solve the equation
t, U = crank_nicolson(a, r, k, T, U0, N)

# plot the solution
plt.plot(t, U)
plt.xlabel('t')
plt.ylabel('U')
plt.title('Solution of the equation')
plt.show()



ValueError: setting an array element with a sequence.

## Application of the SIR model
We first create a simple SIR model using the model described in p1.pdf. The ODE is solved using the scipy library.

In [ ]:
# Defining the model in form of ODE
def SIR_ODE(SIR, t, Beta, Gamma):
    """
    Input:
    SIR: list of 3 elements, S, I, R
    t: time
    Beta: transmission rate
    Gamma: recovery/death rate

    return:
    list of diffrentials of S, I and R
    """
    S, I, R = SIR
    dS =  - Beta*S*I
    dI = Beta*S*I - Gamma*I
    dR = Gamma*I
    return [dS, dI, dR]

t = np.arange(0, 100, 1)

N = 10000
I = 1/N
S = (N - 1)/N
R = 0

Beta0, Gamma0 = 3, 1

#Solving the ODE 
solution = scipy.integrate.odeint(SIR_ODE, y0 =[S, I, R], t=t, args=(Beta0, Gamma0) )


In [ ]:

plt.plot(t, solution[:,0], label='S')
plt.plot(t, solution[:,1], label='I')
plt.plot(t, solution[:,2], label='R')
plt.plot(t, solution[:,0] + solution[:,1] + solution[:,2], label='N')
plt.legend()
plt.xlabel('time')
plt.ylabel('Percent of population')
plt.show()

## Attempting multiple intitial values

Now that we have a working model, we generalize it into a function, then attempt different initial conditions 

In [ ]:
def simulate(I0, Beta0, Gamma0, N= 10000):
    t = np.arange(0, 100, 1)

    I = I0/N
    S = (N - I0)/N
    R = 0

    solution = scipy.integrate.odeint(SIR_ODE, y0=[S, I, R], t=t, args=(Beta0, Gamma0) )
    plt.plot(t, solution[:,0], label='S')
    plt.plot(t, solution[:,1], label='I')
    plt.plot(t, solution[:,2], label='R')
    plt.plot(t, solution[:,0] + solution[:,1] + solution[:,2], label='N')
    plt.legend()
    plt.xlabel('time')
    
    plt.ylabel('Percent of population')
    plt.title('Beta = ' + str(Beta0) + ' Gamma = ' + str(Gamma0) + ' I_0 = ' + str(I0))
    plt.show()

In [ ]:

for i in range(1, 100, 30):
    for g in range(1, 10, 3):
        for b in range(1, 10, 3):
            simulate(i, b, g, N=100)

# Studying the spread of disease in space

In [ ]:
mu_s = 0.01
mu_i = 0.01
Beta0 = 3
Gamma0 = 1



M=50
N=500
k = 1/N
h = 1/M

def scheme_S(S, I, i, j, n, mu=mu_s, Beta=Beta0, k=k, h=h):
    if  k*mu/h**2>0.25:
        print( "stability condition not met")
    return (k*mu/h**2) *(S[i-1,j, n-1] + S[i+1,j,n-1] + S[i,j-1,n-1] + S[i,j+1,n-1]) + (1-k*Beta*I[i,j,n-1]- 4*k*mu/h**2) * S[i,j,n-1]


def scheme_I(S, I, i, j, n, mu=mu_i, Beta=Beta0, k=k, h=h, Gamma=Gamma0):
    if  k*mu/h**2>0.25:
        print( "stability condition not met")
    return I[i,j,n-1]*(k*Beta*S[i,j,n-1] - k*Gamma + 1 - 4*k*mu/h**2) + (k*mu/h**2)*(I[i-1,j,n-1] + I[i+1,j,n-1] + I[i,j-1,n-1] + I[i,j+1,n-1])


S = np.ones((M, M, N))
I = np.zeros((M, M, N))

#Removing the boundary points
S[0,:,:] = 1
S[-1,:,:] = 1 
S[:,0,:] = 1
S[:,-1,:] = 1

#Removing the boundary points
I[0,:,:] = 0
I[-1,:,:] = 0
I[:,0,:] = 0
I[:,-1,:] = 0

# Adding a small number of infected people to the system
I[10:25,10:25,0] = 0.1
S = S - I




for n in range(1,N):
    for i in range(1,M-1):
        for j in range(1,M-1):

            S[i,j,n] = scheme_S(S, I, i,j,n)#(k*mu_s/h**2) *(S[i-1,j, n-1] + S[i+1,j,n-1] + S[i,j-1,n-1] + S[i,j+1,n-1]) + (1-k*Beta0*I[i,j,n-1]- 4*k*mu_s/h**2) * S[i,j,n-1] 
            I[i,j,n] = scheme_I(S, I, i,j,n)#I[i,j,n-1]*(k*Beta0*S[i,j,n-1] - k*Gamma0 + 1 - 4*k*mu_i/h**2) + (k*mu_i/h**2)*(I[i-1,j,n-1] + I[i+1,j,n-1] + I[i,j-1,n-1] + I[i,j+1,n-1])

R = 1 - S - I





In [ ]:
for i in range(0, N, round(N/10)):
    plt.imshow(S[:,:,i], vmin=0, vmax=1)
    plt.title(f'S at time {i}')
    plt.colorbar()
    plt.grid()
    plt.show()
    
    
    plt.imshow(I[:,:,i], vmin=0, vmax=1)
    plt.title(f'I at time {i}')
    plt.colorbar()
    plt.grid()
    plt.show()

## Saving the plots as gif's to better display them

This part is quite computing intensive so avoid running them too much

In [ ]:
images = []

for i in range(0, N, round(N/100)):
    plt.imshow(S[:,:,i], vmin=0, vmax=1)
    plt.title(f'Susceptible people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('S.png')
    images.append(imageio.imread('S.png'))
    plt.close()
imageio.mimsave('S.gif', images)

images = []

for i in range(0, N, round(N/100)):
    plt.imshow(I[:,:,i], vmin=0, vmax=1)
    plt.title(f'Infected people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('I.png')
    images.append(imageio.imread('I.png'))
    plt.close()

imageio.mimsave('I.gif', images)

images = []
for i in range(0, N, round(N/100)):
    plt.imshow(R[:,:,i], vmin=0, vmax=1)
    plt.title(f'Removed people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('R.png')
    images.append(imageio.imread('R.png'))
    plt.close()

imageio.mimsave('R.gif', images)



In [ ]:
Image(filename="S.gif")

In [ ]:
Image(filename="I.gif")

In [ ]:
Image(filename="R.gif")


In [ ]:
# Define the sceme for the SIR model
def simulate_infections(S, I, mu_s, mu_i, Beta0, Gamma0, M, N, k, h):
    for n in range(1,N):
        for i in range(1,M-1):
            for j in range(1,M-1):
                S[i,j,n] = scheme_S(S, I, i, j, n, mu_s, Beta0, k, h)
                I[i,j,n] = scheme_I(S, I, i, j, n, mu_i, Beta0, k, h, Gamma0)

    R = 1 - S - I
    return S, I, R

def plot_infections(S, I, T):
    for t in range(0, T, round(T/5)):
        plt.imshow(S[:,:,t], vmin=0, vmax=1)
        plt.title(f'S at time {t}')
        plt.colorbar()
        plt.grid()
        plt.show()

        plt.imshow(I[:,:,t], vmin=0, vmax=1)
        plt.title(f'I at time {t}')
        plt.colorbar()
        plt.grid()
        plt.show()
        


## Using an infected boundary 

In [ ]:

S = np.ones((M, M, N))
I = np.zeros((M, M, N))

#Removing the boundary points
S[0,:,:] = 0
S[-1,:,:] = 0
S[:,0,:] = 0
S[:,-1,:] = 0

#Infecting the boundary points
I[0,:,:] = 1
I[-1,:,:] = 1
I[:,0,:] = 1
I[:,-1,:] = 1

S, I, R = simulate_infections(S, I, mu_s, mu_i, Beta0, Gamma0, M, N, k, h)
plot_infections(S, I, N)


# Trying out different intial contitions

In [ ]:
mu_s = 0.01
mu_i = 0.001
Beta0 = 3
Gamma0 = 1

M=50
N=500
k = 1/N
h = 1/M

#Random intial infection
S = np.random.randint(9,11, (M,M,N))/10
I = 1 - S

#Removing the boundary points
S[0,:,:] = 0
S[-1,:,:] = 0
S[:,0,:] = 0
S[:,-1,:] = 0

#Removing the boundary points
I[0,:,:] = 0
I[-1,:,:] = 0
I[:,0,:] = 0
I[:,-1,:] = 0

S, I, R = simulate_infections(S, I, mu_s, mu_i, Beta0, Gamma0, M, N, k, h)
plot_infections(S, I, N)


In [ ]:
images = []

for i in range(0, N, round(N/100)):
    plt.imshow(S[:,:,i], vmin=0, vmax=1)
    plt.title(f'Susceptible people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('S.png')
    images.append(imageio.imread('S.png'))
    plt.close()
imageio.mimsave('S.gif', images)

images = []

for i in range(0, N, round(N/100)):
    plt.imshow(I[:,:,i], vmin=0, vmax=1)
    plt.title(f'Infected people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('I.png')
    images.append(imageio.imread('I.png'))
    plt.close()

imageio.mimsave('I.gif', images)

Image(filename="S.gif")


In [ ]:
Image(filename="I.gif")

In [ ]:
for i in range(0, N, round(N/10)):
    plt.imshow(R[:,:,i], vmin=0, vmax=1)
    plt.title(f'R at time {i}')
    plt.colorbar()
    plt.grid()
    plt.show()

# Simulating people in one area not washing hands

In [ ]:
# Define the sceme for the SIR model
def simulate_dirtyhands(S, I, mu_s, mu_i, Beta0, Beta_dh, Gamma0, M, N, k, h):
    for n in range(1,N):
        for i in range(1,M-1):
            for j in range(1,M-1):
                if i<10 and j < 10:
                    S[i,j,n] = scheme_S(S, I, i, j, n, mu_s, Beta_dh, k, h)
                    I[i,j,n] = scheme_I(S, I, i, j, n, mu_i, Beta_dh, k, h, Gamma0)
                else:
                    S[i,j,n] = scheme_S(S, I, i, j, n, mu_s, Beta0, k, h)
                    I[i,j,n] = scheme_I(S, I, i, j, n, mu_i, Beta0, k, h, Gamma0)

    R = 1 - S - I
    return S, I, R

N=1000

#Random intial infection
S = np.random.randint(9,11, (M,M,N))/10
I = 1 - S

#Setting the boundary to be healthy
S[0,:,:] = 1
S[-1,:,:] = 1
S[:,0,:] = 1
S[:,-1,:] = 1

#Removing the boundary points
I[0,:,:] = 0
I[-1,:,:] = 0
I[:,0,:] = 0
I[:,-1,:] = 0

S,I, R = simulate_dirtyhands(S, I, mu_s, mu_i, Beta0, 60, Gamma0, M, N, k, h)

In [ ]:
plot_infections(S, I, N)


In [ ]:
images = []

for i in range(0, N, round(N/100)):
    plt.imshow(S[:,:,i], vmin=0, vmax=1)
    plt.title(f'Susceptible people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('S.png')
    images.append(imageio.imread('S.png'))
    plt.close()
imageio.mimsave('S_dh.gif', images)

images = []

for i in range(0, N, round(N/100)):
    plt.imshow(I[:,:,i], vmin=0, vmax=1)
    plt.title(f'Infected people at time {i}')
    plt.colorbar()
    plt.grid()
    plt.savefig('I.png')
    images.append(imageio.imread('I.png'))
    plt.close()

imageio.mimsave('I_dh.gif', images)



In [ ]:
#Show both gifs in subplots at the same time

Image(filename="S_dh.gif")

In [ ]:
Image(filename="I_dh.gif")

